In [4]:
import accelerate 
import transformers
import json

REPO_ID = "meta-llama/Llama-2-7b-chat-hf"

# Make sure you set this variable according to the environment
environment = "local"
# Show that when we do this neither GPU nor CPU memory increases
config = transformers.AutoConfig.from_pretrained(REPO_ID)

with accelerate.init_empty_weights():
    fake_model = transformers.AutoModelForCausalLM.from_config(config)
    
device_map = accelerate.infer_auto_device_map(fake_model, max_memory={0: "3GiB", "cpu": "6GiB"})
#print(json.dumps(device_map, indent=4))


tokenizer = transformers.LlamaTokenizer.from_pretrained(REPO_ID)
model = transformers.LlamaForCausalLM.from_pretrained(
        REPO_ID, 
        device_map=device_map, 
        offload_folder="/tmp/.offload",
        load_in_8bit=True,
        llm_int8_enable_fp32_cpu_offload=True,
    )

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/yuexichen/opt/anaconda3/lib/python3.9/site-packages/huggingface_hub/utils/_errors.py", line 239, in hf_raise_for_status
    Raises when the request has failed:
  File "/Users/yuexichen/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/yuexichen/opt/anaconda3/lib/python3.9/site-packages/transformers/utils/hub.py", line 409, in cached_file
    cache_dir = TRANSFORMERS_CACHE
  File "/Users/yuexichen/opt/anaconda3/lib/python3.9/site-packages/huggingface_hub/utils/_validators.py", line 124, in _inner_fn
    """Validate `repo_id` is valid.
  File "/Users/yuexichen/opt/anaconda3/lib/py

In [ ]:
batch = tokenizer(
    "I would explain a blackhole to a 5 year old as",
    return_tensors="pt", 
    add_special_tokens=False
)

# /home/thushv89/anaconda3/envs/ml.torch/lib/python3.9/site-packages/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on meta. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('meta') before running `.generate()`.
batch = {k: v for k, v in batch.items()}
n_input_tokens = batch["input_ids"].shape[-1]

t1 = time.perf_counter()
generated = model.generate(batch["input_ids"].to("cuda"), max_length=n_input_tokens+25)
t2 = time.perf_counter()
print(tokenizer.decode(generated[0]))
n_generated = generated.shape[-1]-batch["input_ids"].shape[-1]
print(f"It took {t2-t1}s to generate the sequence of {n_generated} tokens ({n_generated/(t2-t1)} tokens/s).")